<a href="https://colab.research.google.com/github/Vagueken/Visual-Emotion-Detection/blob/main/TL_2_EfficientNET_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Face Emotion Recognition with Transfer Learning. Face recognition with EfficientNetB2 :

In this project we have taken a kaggle dataset to classify some human emotions.

The main motive of this project was to use the SOTA model produced by GOOGLE AI EfficientNet : EfficientNetB2.

In [ ]:
import os
import cv2
import math
import numpy as np
import pandas as pd

%pip install scikit-plot 
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAvgPool2D, GlobalMaxPool2D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.utils import plot_model

from keras.utils import np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.layers import Lambda,Input,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB2

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
# choose the kaggle.json file that you downloaded
files.upload()
! mkdir ~/.kaggle

Saving kaggle.json to kaggle.json


In [ ]:
# make a directory named kaggle and copy the kaggle.json file there
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 80% 96.0M/121M [00:00<00:00, 249MB/s]
100% 121M/121M [00:00<00:00, 230MB/s] 


In [ ]:
from zipfile import ZipFile

zf = ZipFile('face-expression-recognition-dataset.zip', 'r')
zf.extractall('../input/face-expression-recognition-dataset/')
zf.close()

In [ ]:
picture_size = 48
folder_path = "../input/face-expression-recognition-dataset/images/"

In [ ]:
IMAGE_SIZE=[224,224]
train_path= folder_path+'images/train'

valid_path=folder_path+'images/validation'

In [ ]:
eff=EfficientNetB2(input_shape=IMAGE_SIZE+[3],include_top=False)

In [ ]:
for layer in eff.layers:
  layer.trainable=False

In [ ]:
folders=glob('../input/face-expression-recognition-dataset/images/train/*')

In [ ]:
folders

['../input/face-expression-recognition-dataset/images/train/happy',
 '../input/face-expression-recognition-dataset/images/train/disgust',
 '../input/face-expression-recognition-dataset/images/train/fear',
 '../input/face-expression-recognition-dataset/images/train/surprise',
 '../input/face-expression-recognition-dataset/images/train/angry',
 '../input/face-expression-recognition-dataset/images/train/neutral',
 '../input/face-expression-recognition-dataset/images/train/sad']

In [ ]:
x=Flatten()(eff.output)

We just want to change the last layer as we want to classify the emotions with desirable classes.

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)

model=Model(inputs=eff.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      

We can see the model is huge. So we need a lot of time to run it.

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Now making the datasets for training and testing.

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory(train_path,
                                               target_size=(224,224),batch_size=32,class_mode='categorical')

Found 28821 images belonging to 7 classes.


In [ ]:
test_set=test_datagen.flow_from_directory(valid_path,
                                               target_size=(224,224),batch_size=32,class_mode='categorical')

Found 7066 images belonging to 7 classes.


In [ ]:
ef=model.fit_generator(training_set,validation_data=test_set,
                      epochs=3,
                      steps_per_epoch=len(training_set),
                      validation_steps=len(test_set))

<ipython-input-25-c564c64de466>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  ef=model.fit_generator(training_set,validation_data=test_set,


Epoch 1/3
901/901 [==============================] - 353s 391ms/step - loss: 5.2475 - accuracy: 0.1777 - val_loss: 2.6415 - val_accuracy: 0.1611
Epoch 2/3
901/901 [==============================] - 350s 389ms/step - loss: 5.1976 - accuracy: 0.1736 - val_loss: 7.9394 - val_accuracy: 0.1359
Epoch 3/3
901/901 [==============================] - 351s 390ms/step - loss: 4.6755 - accuracy: 0.1792 - val_loss: 3.9917 - val_accuracy: 0.1441


In [ ]:
#Fetching accuracy and loss of test and train dataset
score = model.evaluate(test_set,verbose=0)
print('Test accuracy:', score[1])

score = model.evaluate(training_set,verbose=0) 
print('Train accuracy:', score[1])


Test accuracy: 0.14407019317150116
Train accuracy: 0.14236147701740265


This takes a lot of time so i ran it for just 3 epochs.

and it seems that if you run it for 10 epochs you may get more than 90% accuracy on the validation set. 
